In [13]:
%load_ext autoreload
%autoreload 2
from causalboundingengine.algorithms.conf.tianpearl import TianPearl
from causalboundingengine.algorithms.conf.manski import Manski
from causalboundingengine.scenarios import BinaryConf
import numpy as np
import logging
logging.basicConfig(level=logging.WARNING)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
X = np.array([0, 1, 1])  # Example treatment data
Y = np.array([0, 0, 0])  # Example outcome data


alg = TianPearl()  # Instantiate the Manski algorithm

alg.bound_ATE(X, Y)  # Example usage, replace X and Y with actual data

(np.float64(-1.0), np.float64(1.0))

In [21]:
scenario = BinaryConf(X, Y)
scenario.PNS.entropybounds(theta=.9)  # Example usage with theta parameter

(np.float64(0.0), np.float64(0.5678654629382529))